## Deep Research with multi agents

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool 
from IPython.display import Markdown
import os


def llm(provider: str="groq", model: str="meta-llama/llama-4-maverick-17b-128e-instruct"):
    llm = LLM(model=f"{provider}/{model}")
    return llm

def deep_search(topic):
    # 1. Agents
    researcher_agent = Agent(
        role = "Senior Researcher of {topic}",
        goal = "Perform a deep research on the topic {topic} and provide accurate information on it",
        backstory="You deeply research {topic} and give an accurate, concise result."
        "You base your research on the outline of the topic."
        "Your research is scientific and based one critical thinking."
        "You don't invent anything, rather you provide the result of the search." \
        "Indeed, You check very well the source of the information to avoid fallacies.",
        # Tools = [WebsiteSearchTool],
        allow_delegation= False,
        llm = llm(),
    )
    writer_agent = Agent(
        role="Content writer specialist",
        goal = "",
        backstory = "You're writing on {topic}."
        "You base your writing on the work of "
              "the Senior Researcher, who provides the result of "
              "relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Senior Researcher. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Senior Researcher. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
        # Tools = [WebsiteSearchTool],
        allow_delegation = False,
        llm = llm("groq", "moonshotai/kimi-k2-instruct"),
        
    )
    editor_agent = Agent(
        role="Editor specialist",
        goal = "Edite a given blog post to align with the writing "
        "style of the field or the organization.",
        backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows either scientific research best practices" \
              "or journalistic best practices, that"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
        allow_delegation=False,
        llm = LLM(model="groq/moonshotai/kimi-k2-instruct"),
    )

    # 2. Tasks
    researcher_task= Task(
        description= (
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points,"
            "and a call to action. \n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
                    "with an outline, audience analysis "
                    "SEO keywords, and resources",
        agent = researcher_agent,
        # tools= [],
    )
    writer_task= Task(
        description=(
        "1. Use the research result to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
        ),
        expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
        # tools= [],
        agent = writer_agent,
    )
    editor_task = Task(
        description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
        expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
        # Tools=[],
        agent= editor_agent,
    )

    # 3. Crew
    crew = Crew(
        agents= [researcher_agent, writer_agent, editor_agent],
        tasks=[researcher_task, writer_task, editor_task],
        process = Process.sequential,
        memory = True,
    )

    result = crew.kickoff(inputs= {"topic": topic})
    # return display(Markdown(result.raw))
    # return Markdown(result)
    return result.raw     # work with gr.Textbox() and gr.Markdown()
    # return result


# if __name__=="__main__":
#     deep_search("AGI")

In [42]:
import gradio as gr

with gr.Blocks() as app:
    gr.Markdown("# Deep Research")
    with gr.Row(scale=7):
        with gr.Column(scale=5):
            subject = gr.Textbox(label="Research Subject", lines=3)
        with gr.Column(scale=1, min_width=1):
            search_btn = gr.Button("DeepSearch")
    with gr.Row():
        # result = gr.Textbox(label="Research Result")
        result = gr.Markdown()
    
    search_btn.click(
        fn = deep_search,
        inputs=[subject],
        outputs=[result]
    )
    
app.launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.

🔨 MCP server (using SSE) running at: http://127.0.0.1:7887/gradio_api/mcp/sse


2025-07-29 18:10:24,579 - 135515792266816 - rag_storage.py-rag_storage:132 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-29 18:10:27,619 - 135515792266816 - rag_storage.py-rag_storage:132 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-29 18:10:35,162 - 135515792266816 - rag_storage.py-rag_storage:103 - ERROR: Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
/home/crs/10Academy/llm-projects/multiAgent/.venv/lib/python3.10/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
2025-07-29 18:10:39,673 - 135515792266816 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only ar

In [36]:
from IPython.display import Markdown
import gradio as gr

def greet(name):
    result = f""" # Hello {name},\n Welcome to the real world.\n 
    - *AI Startup*
    - *Trading Bots powered AI*
    """
    # return display(Markdown(result))  # display in notebook as markdown
    return result

# app.gr.Interface(
#     fn = greet,
#     inputs= [gr.Textbox(label="Your name")],
#     outputs= [gr.Textbox()]
# )

with gr.Blocks() as app:
    input = gr.Textbox(label= "Your name")
    # output= gr.Textbox()
    output = gr.Markdown()
    btn = gr.Button("Submit")

    btn.click(fn=greet, inputs=[input], outputs=[output])
app.launch()

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.

🔨 MCP server (using SSE) running at: http://127.0.0.1:7884/gradio_api/mcp/sse
